In [7]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [8]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""You are an expert editor focused on clarity and simplicity. 
            Your job is to eliminate ambiguity, redundancy, and make every sentence crisp and clear. 
            Don't worry about persuasion or tone — just make the message easy to read and understand.""",
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""You are a communication coach focused on emotional tone and professionalism. 
            Your job is to make the email sound warm, confident, and human — while staying professional 
            and appropriate for the audience. Improve the emotional resonance, polish the phrasing, 
            and adjust any words that may come off as stiff, cold, or overly casual.""",
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""You are a persuasion expert trained in marketing, behavioral psychology, 
            and copywriting. Your job is to enhance the email's persuasive power: improve call to action, structure arguments, and emphasize benefits. Remove weak or passive language.""",
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""You are an advanced email-writing specialist. Your role is to read all 
            prior agent responses and revisions, and then **synthesize the best ideas** into a unified, 
            polished draft of the email. Focus on: Integrating clarity, tone, and persuasion improvements; 
            Ensuring coherence, fluency, and a natural voice; Creating a version that feels professional, 
            effective, and readable.""",
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""You are an email quality evaluator. Your job is to perform a final review 
            of the synthesized email and determine if it meets professional standards. Review the email for: 
            Clarity and flow, appropriate professional tone, effective call-to-action, and overall coherence.
            Be constructive but decisive. If the email has major flaws (unclear message, unprofessional tone, 
            or missing key elements), provide ONE specific improvement suggestion. If the email meets professional standards and communicates effectively, respond with 'The email meets professional standards.' followed by `TERMINATE` on a new line. You should only approve emails that are perfect enough for professional use, dont settle.""",
)

In [9]:
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_messages_termination

In [10]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="Hi! Im hungry, buy me lunch and invest in my business. Thanks."
    )
)

---------- TextMessage (user) ----------
Hi! Im hungry, buy me lunch and invest in my business. Thanks.
---------- TextMessage (ClarityAgent) ----------
Hi! I'm hungry. Please buy me lunch and invest in my business. Thank you.
---------- TextMessage (ToneAgent) ----------
Subject: A Quick Favor

Hi there!

I hope you're doing well! I find myself feeling a bit peckish today, and I would truly appreciate it if you could treat me to lunch. Additionally, if you're open to it, I would love to discuss an opportunity to invest in my business. Your support could make a significant difference!

Thank you so much for considering this. Looking forward to your response!

Warm regards,  
[Your Name]
---------- TextMessage (PersuasionAgent) ----------
Subject: Let's Make Delicious Opportunities Happen!

Hi [Recipient's Name],

I hope you're having a fantastic day! I wanted to reach out to ask for your help with two exciting opportunities.

First, I'm feeling a bit hungry today, and I would greatly a

TaskResult(messages=[TextMessage(id='b44b4fcd-33e4-4b3e-b14c-ed35f4e6cff5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 13, 32, 30, 688581, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, buy me lunch and invest in my business. Thanks.', type='TextMessage'), TextMessage(id='959c40ae-618e-4a8f-8ae2-4f494d430140', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=77, completion_tokens=18), metadata={}, created_at=datetime.datetime(2025, 8, 7, 13, 32, 31, 531234, tzinfo=datetime.timezone.utc), content="Hi! I'm hungry. Please buy me lunch and invest in my business. Thank you.", type='TextMessage'), TextMessage(id='bc369cf4-cc33-4206-b077-ceacbe7b68ab', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=123, completion_tokens=91), metadata={}, created_at=datetime.datetime(2025, 8, 7, 13, 32, 33, 550333, tzinfo=datetime.timezone.utc), content="Subject: A Quick Favor\n\nHi there!\n\nI hope you're doing well! I find mysel